<a href="https://colab.research.google.com/github/mmistroni/jupyter/blob/master/StockAndNewsAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install pandas-datareader
!pip install nltk
!pip install vaderSentiment

In [ ]:
!pip install -U -q PyDrive

In [ ]:
import urllib
import json
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime, date
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request


In [ ]:
import pandas as  pd
from datetime import date
import logging
from pandas.tseries.offsets import BDay
import requests

def get_data(ticker, dt, busdays=1):
  today = date.today()
  start_date = (today- BDay(busdays))
  full_url = 'https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={start_date}&to={end_date}'\
                    .format(ticker=ticker, start_date=start_date.strftime('%Y-%m-%d'), end_date=today.strftime('%Y-%m-%d'))
  return requests.get(full_url).json()

def get_latest_price_yahoo_dataflow(ticker, bday=1):
  from datetime import date
  today = date.today().strftime('%Y-%m-%d')
  start_date = today- BDay(bday)
  today_df = get_data(ticker, today)
  yday_df = get_data(ticker, start_date).strftime('%Y-%m-%d')
  yday_df = yday_df.rename(columns={"Adj Close": "Prev Close", "Volume": "Prev Volume"})
  merged = pd.merge(today_df, yday_df, on='Symbol')
  merged['Diff'] = merged['Adj Close'] - merged['Prev Close']
  merged['Vol Diff'] = merged['Volume'] - merged['Prev Volume']
  return merged
  

<h2>Authenticate User </h2>

In [ ]:
from google.colab import auth
auth.authenticate_user()




<h3>Loading Credentials</h3>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

def get_iexapi_keys():
  with open('gdrive/My Drive/passwords/iexapi.keys') as f:
    return f.readlines()[0]

def get_nlp_service_keys():
  with open('gdrive/My Drive/passwords/nlp.keys') as f:
    return f.readlines()[0]

def get_newsapi_keys():
  with open('gdrive/My Drive/passwords/newsapi.keys') as f:
    return f.readlines()[0]

def getfmpkeys():
  with open('gdrive/My Drive/passwords/fmprep.keys') as f:
    return f.readlines()[0]







In [ ]:
def get_prices(ticker):
    iexapikey = get_iexapi_keys()
    iexurl = 'https://cloud.iexapis.com/stable/stock/{ticker}/quote?token={token}'.format(
                                ticker=ticker, token=iexapikey)
    return requests.get(iexurl).json()


def get_all_prices():
  all_stocks = requests.get('https://financialmodelingprep.com/api/v3/company/stock/list?apikey={}'.format(getfmpkeys())).json()['symbolsList']
  tickers = list(map(lambda d: d['symbol'], all_stocks))
  for t in tickers:
    try:
      get_prices(t)
    except Exception as e:
      print('unable to find {}:{}'.format(t, str(e)))


<h3>IEX API CALLS </h3>


In [ ]:
import requests

token = get_iexapi_keys()

def get_statistics(ticker):
  base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={token}&format=csv&filter=companyName,symbol,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(token=token,symbol=ticker)
  df = pd.read_csv(base_url)
  df['Symbol'] = ticker
  return df

def get_iex_historical_data(ticker, nummonths):
  return requests.get('https://cloud.iexapis.com/stable/stock/{ticker}/chart/{months}m?token={token}&chartCloseOnly=true'.format(token=token,
                                                                                                                          ticker=ticker,
                                                                                                                          months= nummonths)).json()
  

def get_all_stocks():
  all_symbols_data = requests.get('https://cloud.iexapis.com/stable/ref-data/iex/symbols?token={token}'.format(token=token)).json()
  return [d['symbol'] for d in all_symbols_data if d['isEnabled']and d['type'].lower() == 'cs']

def get_all_us_stocks(security_type='cs', nasdaq=False):
  nyse_symbols = requests.get('https://cloud.iexapis.com/stable/ref-data/exchange/nys/symbols?token={token}'.format(token=token)).json()
  nas_symbols = requests.get('https://cloud.iexapis.com/stable/ref-data/exchange/nas/symbols?token={token}'.format(token=token)).json()
  all_symbols = nyse_symbols + nas_symbols if nasdaq else nyse_symbols
  return [d['symbol'] for d in all_symbols  if d['type'].lower() == security_type]

def get_all_etfs():
  return get_all_us_stocks(security_type='et', nasdaq=True)
  
def get_all_stocks_data():
  good_ones = get_all_etfs()
  return map(lambda symbol: (symbol, get_historical_value(symbol)), good_ones)


def get_all_exchanges():
  return requests.get('https://cloud.iexapis.com/stable/ref-data/market/us/exchanges?token={token}'.format(token=token)).json()

def get_latest_price(symbol):
  base_url = "https://cloud.iexapis.com/stable/stock/{ticker}/quote?token={token}&format=csv&filter=symbol,close".format(token=token,ticker=symbol)
  import requests
  return pd.read_csv(base_url)

def get_quote(symbol):
  try:
    historical_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/quote/latestPrice?token={token}'.format(token=token,symbol=symbol)
    return requests.get(historical_url).json()
  except:
    return -1

def get_news(symbol, num_of_news):
  try:
    news_url = 'https://cloud.iexapis.com/stable//stock/{symbol}/news/last/{last}?token={token}'.format(symbol=symbol, last=num_of_news,
                                                                                                        token=token)
    return requests.get(news_url).json()
  except Exception as e :
    print('Excepiton for {}:{}'.format(symbol, str(e)))
    return []

def get_prices(ticker, iexkey):
  try:
    stat_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={token}'.format(symbol=ticker, token=iexkey)
    return requests.get(stat_url).json()
  except Exception as e :
    print('Excepiton for {}:{}'.format(symbol, str(e)))
    return []




<h3> Yahoo API Calls </h3>

In [ ]:
from scipy.stats import pearsonr
import requests

def get_latest_price_yahoo(symbol, as_of_date):
  try:#
    print('--latest price for{}'.format(symbol))
    res = dr.get_data_yahoo(symbol, as_of_date, as_of_date)[['Close']]
    df['Symbol'] = symbol
    return df
  except Exception as e :
    return pd.DataFrame(columns=[symbol])

def get_historical_data_yahoo(symbol, start_dt, end_dt):
  try: 
    end_date = date.today()
    data = dr.get_data_yahoo(symbol, start_dt, end_dt)[['Adj Close']]
    df =  data.rename(columns={'Adj Close' : symbol})
    return df
  except Exception as e :
    return pd.DataFrame(columns=[symbol])   
  

<h3> Getting Sentiment Analysis from Vadder </h3>

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# This is good one, Free

def get_sentiment_from_vader(sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    return score['compound']

get_sentiment_from_vader("Here's what booming dollar store sales say about America right now Yahoo Finance")




<h3> Getting Sentiment Analysis from Google </h3>

In [ ]:
def get_sentiment_from_google(content):
  from google.cloud import language
  from google.cloud.language import enums
  from google.cloud.language import types

  client = language.LanguageServiceClient()
  document = types.Document(
      content=clean_text,
      type=enums.Document.Type.PLAIN_TEXT)

  # Detects the sentiment of the text
  sentiment = client.analyze_sentiment(document=document).document_sentiment

  



<h3> Google Language API </h3>

In [ ]:
def test_language_api(content):
  import httplib2
  import sys
  from googleapiclient import discovery
  from googleapiclient.errors import HttpError

  discovery_url = 'https://{api}.googleapis.com/$discovery/rest?version={apiVersion}'

  service = discovery.build(
      'language', 'v1',
      http=httplib2.Http(),
      discoveryServiceUrl=discovery_url,
      developerKey=get_nlp_service_keys(),
  )
  service_request = service.documents().annotateText(
      body={
          'document': {
              'type': 'PLAIN_TEXT',
              'content': content,
          },
          'features': {
              'extract_syntax': True,
              'extractEntities': True,
              'extractDocumentSentiment': True,
          },
          'encodingType': 'UTF16' if sys.maxunicode == 65535 else 'UTF32',
      })
  try:
      #print('************************')
      #print('Retrieving sentiment for:{}'.format(content))
      response = service_request.execute()
      return  response['documentSentiment']['score']
      
  except HttpError as e:
      response = {'error': e}
      print('exception:{}'.format(str(e)))
      return 0


<h3> Getting News from News API <h3>

In [ ]:
def extract_from_news_table(news_tables):
  parsed_news = []
  print('Found:{}'.format(len(news_tables.items())))
  for file_name, news_table in news_tables.items():
      for x in news_table.findAll('tr'):
      
          text = x.a.get_text() 
          date_scrape = x.td.text.split()

          if len(date_scrape) == 1:
              time = date_scrape[0]
              
          else:
              date = date_scrape[0]
              time = date_scrape[1]
          ticker = file_name.split('_')[0]
          
          parsed_news.append([ticker, date, time, text])
  



def retrieve_all_news(symbol):
  print('Retrieving all news for@{}'.format(symbol))
  token = get_newsapi_keys()
  all_news = 'https://newsapi.org/v2/everything?q={ticker}&apiKey={token}'.format(ticker=symbol, token=token)
  data = requests.get(all_news).json()
  res = data['articles']
  return map(lambda data: data['content'], res)

def get_news_from_finviz(tickers):
  news_tables = {}
  for ticker in tickers:
    url = 'https://finviz.com/quote.ashx?t={}'.format(ticker)
    try:
      req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
      response = urlopen(req)    
      html = BeautifulSoup(response)
      print('Now parsing...')
      news_table = html.find(id='news-table')
      news_tables[ticker] = news_table
    except Exception as e:
      print('culd not fetch data for {}:{}'.format(ticker, str(e)))
    
  return extract_from_news_table(news_tables)




def calculate_sentiment(news_items):
  res = map(lambda item: get_sentiment_from_vader(item), news_items)
  all_news =  list(res)
  pprint('Total:{}'.format(all_news))
  return dict(total=sum(all_news), positive=[i for i in all_news if i > 0], negative=[i for i in all_news if i <0])

def calculate_news_sentiment(ticker):
  latest_news = list(retrieve_all_news(ticker))
  return calculate_sentiment(latest_news)



In [ ]:
!gsutil cp gs://datascience-bucket-mm/all_sectors.csv .
sct = pd.read_csv('all_sectors.csv')
sct.rename({'Unnamed: 0' : 'ticker', 'Sector':'sector'}, axis=1, inplace=True)

cons_dens_ticker = sct[sct['sector'] == 'Consumer Defensive']

cons_dens_ticker.shape

In [ ]:
def find_news_scores_for_ticker(tickers):
  parsed_news = get_news_from_finviz(tickers)
  columns = ['ticker', 'date', 'time', 'headline']
  parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)
  parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date
  
  return parsed_and_scored

  parsed_and_scored_news_grouped = parsed_and_scored_news.groupby(['ticker'], as_index = False).agg({'headline': ''.join}, Inplace=True)
  scores = parsed_and_scored_news_grouped['headline'].apply(get_sentiment_from_vader).tolist()
  scores_df = pd.DataFrame(scores)
  parsed_and_scored_news_result = parsed_and_scored_news_grouped.join(scores_df, rsuffix='_right')
  return parsed_and_scored_news, parsed_and_scored_news_result


tickers = cons_dens_ticker['ticker'].values.tolist()


raw = find_news_scores_for_ticker(tickers)

In [ ]:
# Below does notwork. finviz for ticker has different structure from finviz for general data.
base_url = 'https://www.finviz.com/news.ashx'
req = Request(url=base_url,headers={'user-agent': 'my-app/0.0.1'}) 
response = urlopen(req)    
html = BeautifulSoup(response)
print('Now parsing...')
news_table = html.find(id='news-table')
news_table
#extract_from_news_table(news_table)

<h3> Fetching Data From BigQuery </h3>

In [ ]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

client = bigquery.Client('datascience-projects')

In [ ]:
def aggregate(df):
  print('Aggregating by ticker')
  renamed = df.rename({'TICKER': 'Ticker'}, axis=1)
  return renamed.groupby('Ticker', as_index=False)['COUNT'].sum()\
                              .rename(columns={'sum':'TOTAL_FILINGS'})
  
  #return renamed.groupby('Ticker', as_index=False).COUNT.agg(['sum']).rename({'sum' : 'TOTAL_FILINGS'},axis=1)

def get_edgar_data():
  sql = """
    SELECT *
    FROM
        `datascience-projects.gcp_edgar.form_13hf_daily`
      

    """
  df = client.query(sql).to_dataframe()
  df['asofdate'] = pd.to_datetime(df['COB'], format='%Y-%m-%d')

  print('Only considering last 2 months worth of iling')
  start_date = date.today() - BDay(40)

  filtered = df[df['asofdate']> start_date]
  
  return aggregate(filtered)

In [ ]:
relevant = get_edgar_data()
relevant[relevant['Ticker'].isin(['NIO', 'MGM'])].head(10)

<h3>Computing various  metrics </h3>

In [ ]:
# we need to sort out which function we need.
#1. calculate sharpe ratio  DONE
#2. get month change percent m30
#3. get full performance   DONE
#4. get news  DONE
#5 group together and compute

import logging
logger = logging.getLogger(__name__)
from math import sqrt
from pprint import pprint

def calculate_daily_returns(prices):
  return prices.pct_change(1)

def calculate_daily_cumulative_returns(daily_pc):
  return (1 + daily_pc).cumprod()

def compute_standard_deviation(daily):
  return daily.loc[:,daily.columns[0]].std()

def compute_sharpe_ratio(s_prices):
  # This function should be used in final dataframe
  # USE THSI FUNCTION
  dret = calculate_daily_returns(s_prices)
  avg = dret.loc[:,dret.columns[0]].mean()
  std = compute_standard_deviation(dret)
  return (sqrt(252) * avg) / std

def compute_moving_averages(prices, day):
  return prices.rolling(window=day).mean()

def check_prices_vs_moving_averages(prices, day=30):
  # This Function should be used  in final dataframe
  ma30 = compute_moving_averages(prices, 30)
  ticker_col = ma30.columns[0]
  m30_col = '{}M30'.format(ticker_col)
  ma30_renamed = ma30.rename({ticker_col: m30_col}, axis=1)
  concats = pd.concat([prices, ma30_renamed], axis=1)
  concats['AboveM30'] = concats[ticker_col] > concats[m30_col]
  above_m30 = concats[concats['AboveM30'] == True]
  total_prices = prices.shape[0]
  total_m30 = above_m30.shape[0]
  pcnt = 1.0*total_m30/total_prices
  return pcnt

def compute_data_performance(historical_df, ticker):
  # Use this FUNCTION CALL
  start = historical_df[ticker].values[0]
  end = historical_df[ticker].values[-1]
  return end*1.0/start - 1

def compute_metrics(prices):
  # Add Trade Volumne to spot momentum stocks
  ticker = prices.columns[0]
  perf_dict = {}
  perf_dict['Ticker'] = ticker
  perf_dict['Performance'] = compute_data_performance(prices, ticker)
  perf_dict['Start_Price'] = prices[ticker].values[0]
  perf_dict['End_Price'] = prices[ticker].values[-1]
  #perf_dict['AboveMovingAvgPcnt'] = check_prices_vs_moving_averages(prices, day=15)
  perf_dict['SharpeRatio'] = compute_sharpe_ratio(prices)
  #news_dict =calculate_news_sentiment(ticker)
  #news_measure = sum(news_dict['positive']) + sum(news_dict['negative'])
  #perf_dict['News_Sentiment'] = news_measure
  return pd.DataFrame([perf_dict])


In [ ]:

# get yahoo historical data
def get_data(ticker, dt, busdays=1):
  res = dr.get_data_yahoo(ticker, dt, dt)[['Adj Close', 'Volume']]
  res['Symbol'] = ticker
  return res

def get_share_data(ticker):
  from datetime import date
  today = date.today()
  start_date = today- BDay(1)
  today_df = get_data(ticker, today)   
  print(today_df.head(2))
  yday_df = get_data(ticker, start_date)
  yday_df = yday_df.rename(columns={"Adj Close": "Prev Close", "Volume": "Prev Volume"})
  print('Merging')
  merged = pd.merge(today_df, yday_df, on='Symbol')
  print('Calclating diff')
  merged['Diff'] = merged['Adj Close'] - merged['Prev Close']
  merged['Vol Diff'] = merged['Volume'] - merged['Prev Volume']
  return merged
#compute_metrics(historical_df)
#pd.DataFrame.from_dict(res)



<h3> Reading source data and computing performance </h3>

In [ ]:
from datetime import date
def get_date_ranges(prev_bus_days):
  print('Checking result sfor {}'.format(prev_bus_days))
  end_date = date.today()
  start_date = end_date - BDay(prev_bus_days) # go back 3 months to find crossovers. Start to plan to move to Dataflow
  return start_date, end_date

def compute_performance(idx, start_dt, end_dt, ticker):
  try:
    print('Computing {}'.format(idx))
    # Here we need to compute, in addition to performance, also sharpe ratio etc.
    import time
    historical_df =  get_historical_data_yahoo(ticker, start_dt, end_dt)
    #merged = pd.merge(historical_df, latest_df, how='inner' , on=ticker)
    metrics_df = compute_metrics(historical_df)
    return metrics_df
  except Exception as e:
    print('Exception:{}'.format(str(e)))
    print('Unable to find data for {}:{}'.format(ticker,str(e)))
    
def find_best_performing(start_dt, end_dt, symbols):
  print('Finding Best Performing Stocks between:{}-{}'.format(start_dt, end_dt))
  # Attempting 100 at a time
  # Targeting run on 15/16/17/18 in lot fo 400 at a time, forcing start date to be the friday before so
  # we can cover all news as we have a max request of 500
  
  dfs = (compute_performance(idx, start_dt, end_dt, symbol) for idx, symbol in enumerate(symbols))
  filtered = (df for df in dfs if df is not None)
  all_data = pd.concat(filtered)
  #all_data = all_data[(all_data.Start_Price >= 10)]
  all_data.sort_values(by=['Performance'], inplace=True, ascending=False)
  return all_data
  
#performance_df.to_csv('performance_run_20200207-news-first300.csv')
#!gsutil cp performance_run_20200207-news-first300.csv gs://datascience-bucket-mm/monthly_run/performance_run_20200207-news-first300.csv




<h3> Getting all sectors and indicators datset </h3>

In [ ]:
!mkdir -p data
!gsutil cp gs://datascience-bucket-mm/all_sectors.csv data
sct = pd.read_csv('data/all_sectors.csv')
sct.rename({'Unnamed: 0' : 'ticker', 'Sector':'sector'}, axis=1, inplace=True)
print(sct.shape)



In [ ]:
def compute_performance_for_sector(sector_name, all_sector_df):
  print('Checking perf of  {}'.format(sector_name))
  all_data= []
  s_df = all_sector_df[all_sector_df['sector'] == sector_name]
  all_tickers = s_df.ticker.values.tolist()
  start,end = get_date_ranges(20)
  for idx, ticker in enumerate(all_tickers):
    print(ticker)
    res = compute_performance(idx, start, end, ticker)
    if res is not None:
      all_data.append(res)
  
  if all_data:
    res =  pd.concat([p for p in all_data], axis=0)
    res['sector'] = sector_name
    return res
  else:
    print('===== NO DATA FOUND FOR SECTOR {}={}'.format(sector_name, len(all_tickers)))


In [ ]:
all_sectors = set([s for s in sct.sector.dropna().values.tolist() if s != 'nan'])
first_5to8_dfs = [compute_performance_for_sector(s, sct) for s in list(all_sectors)]

In [ ]:
first_5_res = pd.concat([df for df in first_5to8_dfs if df is not None])
first_5_res.to_csv('august__end_run_all_perf.csv')
first_5_res.shape
!gsutil cp august__end_run_all_perf.csv gs://datascience-bucket-mm/monthly_run/



In [ ]:
!gsutil cp  gs://datascience-bucket-mm/monthly_run/august__end_run_all_perf.csv .

df = pd.read_csv('august__end_run_all_perf.csv')
#df[df['Ticker'].isin(['KR','WST','AMGN','FAST','DXCM','DOCU','GMAB'])]
df.columns

<h3> Get Edgar Data </h3>

In [ ]:
edgar_filing = get_edgar_data().rename({'TICKER':'Ticker'}, axis=1)
edgar_filing.head(2)


<h3> Sector Performance </h3>

In [ ]:
df.groupby(['sector']).mean().sort_values(by=['Performance'], ascending=False)

In [ ]:
aug_shares = df[(df['sector'] == 'Consumer Cyclical') & (df.Start_Price > 10)]


merged = aug_shares.merge(edgar_filing, on='Ticker', how='left').fillna(0)\
           .sort_values(by=['Performance', 'COUNT'], ascending=False).head(30)

merged



In [ ]:
edgar_filing.columns

In [ ]:
start,end = get_date_ranges(20)
compute_performance(0, start, end, 'KR')

<h3> Copying exported files here </h3>

In [ ]:
all_sectors = sct['sector'].dropna().unique().tolist()
print(all_sectors)



<h3> Finding best performers among shares </h3>

In [ ]:
start_dt, end_dt = get_date_ranges(60)
stocks = get_all_us_stocks()
print('We got:{}'.format(len(stocks)))
performance_df = find_best_performing(start_dt, end_dt, stocks[0:10])

#print(performance_df.shape)
performance_df.head(10)
performance_df.to_csv('performance_run_20200306_60_days.csv')
!gsutil cp performance_run_20200302.csv gs://datascience-bucket-mm/monthly_run/performance_run_20200302.csv


<h3> Crssing over with stocks which are still growing

In [ ]:
start_dt, end_dt = get_date_ranges(30)
stocks = get_all_us_stocks()
all_etfs = get_all_etfs()
all_stocks = stocks + all_etfs
print('We got:{}'.format(len(all_stocks)))
performance_df_recent = find_best_performing(start_dt, end_dt, all_stocks)



In [ ]:
#print(performance_df.shape)
sorted_performance = performance_df_recent.sort_values(by=['Performance'], ascending=False)
print(sorted_performance.head(10))
sorted_performance.to_csv('performance_run_20200318_30_days.csv')
!gsutil cp performance_run_20200318_30_days.csv gs://datascience-bucket-mm/monthly_run/performance_run_20200318_30_days.csvs


In [ ]:
print(sorted_performance.head(30))

In [ ]:
find_best_performing(start_dt, end_dt,  ['TVIX'])

In [ ]:
good_60 = performance_df[performance_df['Performance'] > 0]
good_20= performance_df_recent[performance_df_recent['Performance'] > 0]



In [ ]:
merged = good_20.merge(good_60, on='Ticker', how='inner', suffixes=['_20','_60'])

In [ ]:
merged.to_csv('performance_run_20200306_60_vs_20days.csv')
!gsutil cp performance_run_20200306_60_vs_20days.csv gs://datascience-bucket-mm/monthly_run/performance_run_20200306_60_vs_20days.csv


<h3> Finding best performenrs among ETF </h3>

In [ ]:
all_etfs = get_all_etfs()
start_dt, end_dt = get_date_ranges(30)
performance_df_etf_60 = find_best_performing(start_dt, end_dt, all_etfs)


In [ ]:
setf_performance = performance_df_etf_60.sort_values(by=['Performance'], ascending=False)
print(setf_performance.head(20))
setf_performance.to_csv('performance_etf_run_20200318_30_days.csv')
!gsutil cp performance_etf_run_20200318_30_days.csv gs://datascience-bucket-mm/monthly_run/performance_etf_run_20200318_30_days.csvs



In [ ]:
start_dt, end_dt = get_date_ranges(20)
performance_df_etf_20 = find_best_performing(start_dt, end_dt, all_etfs)
good_etf60 = performance_df_etf_60[performance_df_etf_60['Performance'] > 0]
good_etf20= performance_df_etf_20[performance_df_etf_20['Performance'] > 0]
merged_etf = good_etf20.merge(good_etf60, on='Ticker', how='inner', suffixes=['_20','_60'])
merged_etf.to_csv('performance_run_20200306_60_vs_20days_etf.csv')
!gsutil cp performance_run_20200306_60_vs_20days_etf.csv gs://datascience-bucket-mm/monthly_run/performance_run_20200306_60_vs_20days_etf.csv





<h2> TODO: Fetch news for every ticker and find out sentiment. then build a dataframe of symbol, positive news, performance </h3>

<h3> Group by sector, to find best performers </h3>

In [ ]:
res = perf_df[['industry', 'month1ChangePercent','month3ChangePercent', ]].groupby(['industry']).mean().sort_values(by=['month1ChangePercent','month1ChangePercent'], ascending=False)
res.head(20)

<p> Testing all stocks in portfolio </p>

In [ ]:
portfolio_shares = ['ADAC', 'AMBS', 'AMZN', 'AZFL', 'ARSC', 'AAPL', 'APTY',
                    'BTCS', 'BRK-B', 'CRNT', 'CRLBF', 'XOM', 'HAON', 'AGEEF',
                    'HMNY', 'JNJ', 'LEMIF', 'NXTTF', 'NVCN', 'RNVA', 'TORC',
                    'RTRX', 'VALE', 'VZ', 'DGP', 'RUSL', 'REMX', 'TVIX' ]

all_shares = get_all_stocks()
res = map(lambda symbol:(symbol, symbol in all_shares), shares)
invalid = [tpl for tpl in res if not tpl[1]]


<h3> Performance Functions </h3>

In [ ]:
def calculate_correlation_all(vix, all_stocks):
  all_df = [vix]
  res= [vals for _, vals in all_stocks if vals.shape[0] == vix_vals.shape[0]]
  res.append(vix)
  all_data = pd.concat(res, axis=1)
  return all_data.corr('pearson')

def calculate_portfolio_correlation(all_stocks):
  res= [vals for _, vals in all_stocks if vals.shape[0] > 2]
  all_data = pd.concat(res, axis=1)
  return all_data.corr('pearson')


def calculate_correlation(vix, all_stocks):
  result = []
  best = 0
  for symbol, vals in all_stocks:
    if vals.shape[0] == vix.shape[0]:
      concats  = pd.concat([vix, vals], axis = 1)
      corr_matrix = concats.corr(method='pearson')
      corr_with_vix = corr_matrix.loc['^VIX'][1]
      if corr_with_vix > 0 and corr_with_vix > best:
        print('New Corr with {}:{}'.format(symbol, corr_with_vix))
        best = corr_with_vix
  return best

def _get_most_correlated(result_df):
  df = result_df[['^VIX']]
  bad_df = df.index.isin(['^VIX'])
  return df[~bad_df]
  


In [ ]:
#vix_vals = get_historical_value('^VIX')

all_stocks_data = map(lambda symbol: (symbol, get_historical_value(symbol)), portfolio_shares)
best = calculate_portfolio_correlation(all_stocks_data)      


In [ ]:
res = _get_most_correlated(best)
sorted_df = res.sort_values('^VIX', ascending=False)
sorted_df.head(10)


In [ ]:
idx = sorted_df.iloc[0]

print(idx.values.tolist()[0])
print(sorted_df.index[0])

<h3> Getting All US Stocks </h3>

In [ ]:
iexapikey = get_iexapi_keys()
iexurl = 'https://cloud.iexapis.com/stable/stock/market/sector-performance?token={token}'.format(
                              token=iexapikey)
requests.get(iexurl).json()
#TODO we need to find etf tracking different sectors..


<h3> REtrieving News </h3>

<h3> Accessing bigQuery </h3>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
%%bigquery --project datascience-projects --use_legacy_sql --verbose df
SELECT 
  COB, COUNT(*) as total_rows
FROM [datascience-projects.gcp_edgar.form_13hf_data]  GROUP BY COB;

In [ ]:
df

<h3> GEtting quartely reports from IEXAPI </h3>


In [ ]:
def get_last_quartely_statements(ticker, num_statements):
  print('Fetching last {} statements for {}'.format(num_statements, ticker))
  q_url = 'https://cloud.iexapis.com/stable/time-series/REPORTED_FINANCIALS/{}/10-Q?last={}&token={}'.format(ticker, num_statements, get_iexapi_keys())
  print('Loading up {}'.format(q_url))
  return requests.get(q_url).json()



In [ ]:
'ticker' in df.columns.values
df.to_csv('aapl_last_31_10Q_dataset.csv')

In [ ]:
!gsutil cp 'aapl_last_31_10Q_dataset.csv'  gs://mm_dataflow_bucket/inputs

In [ ]:
!gsutil cp gs://datascience-bucket-mm/all_sectors.csv .
sct = pd.read_csv('all_sectors.csv')
sct.rename({'Unnamed: 0' : 'ticker', 'Sector':'sector'}, axis=1, inplace=True)


In [ ]:
from itertools import chain
grouped = sct.groupby('sector').aggregate(lambda tdf: tdf.unique().tolist()[0:5])

sample_tickers = grouped['ticker'].values
all_ticks = list(chain(*sample_tickers))




In [ ]:
grouped.to_csv('random_tickers_for_sector.csv')
!gsutil cp 'random_tickers_for_sector.csv'  gs://mm_dataflow_bucket/inputs


In [ ]:
import pandas as pd

ticker_dict = {}
for ticker in all_ticks:
  latest = get_last_quartely_statements(ticker, 32)
  df = pd.DataFrame(latest)
  print('REsult for {} has shape:{}'.format(ticker, df.shape))
  ticker_dict[ticker] = df



In [ ]:
items = ticker_dict.items()
len(items)

In [ ]:
common_cols_dict = dict()
for k, v in ticker_dict.items():
  if v.shape[0] > 1:
    print('Adding {} with shape {}'.format(k, v.shape))
    common_cols_dict[k] = v
  else:
    print('Skipping {}, its empty'.format(ticker))
  

In [ ]:
all_items_sorted= sorted(common_cols_dict.items(), key=lambda t: t[1].shape[1], reverse=False)

common_cols = all_items_sorted[0][1].columns.values
for k, df in all_items_sorted[1:]:
  current_cols = df.columns.values
  print('({}) - common:{}. current:{}'.format(k, len(common_cols), len(current_cols)))
  itsect = set(common_cols).intersection(set(current_cols))
  r_itsect = set(current_cols).intersection(set(common_cols))
  
  smallest = itsect if itsect < r_itsect else r_itsect
  common_cols = smallest
  print('Now common_cols has:{}'.format(len(common_cols)))

print('Now making sure everyone is ok')
unioned = []
for k, df in all_items_sorted:
  try:
    selected = df[common_cols]
    selected['ticker'] = k
    unioned.append(selected) 
  except Exception as e:
    print('{} exception {}'.format(k, str(e)))

final_df = pd.concat(unioned)

print('Final df  has:{}'.format(final_df.shape))

final_df.to_csv('quarterly_earnings_mixed_tickets_dataset.csv')
!gsutil cp 'quarterly_earnings_mixed_tickets_dataset.csv'  gs://mm_dataflow_bucket/inputs






    
  
    




In [ ]:
print('Now testing if those commoon cols are good enough fo all our datasets :{}'.format(len(common_cols)))
base_cols = common_cols
for ticker, df in ticker_dict.items():
  if df.shape[0] > 0:
    df_cols = df.columns.values
    itsct = set(common_cols).intersection(set(df_cols))
    if len(itsct) < len(base_cols):
      print('UPdating base_cols to {}'.format(len(itsct)))
      base_cols = itsct
    
print('And finally we have {}.Retrying'.format(len(base_cols)))
new_dict = {}
for ticker, df in ticker_dict.items():
  if df.shape[0] > 0:
    print('Testing {} for  common cols'.format(ticker))
    try:
      new_dict[ticker]  = df[base_cols]
    except Exception as e:
      print('{} Exception:{}'.format(ticker, str(e)))
    





